# 🧭 Lecture: Creating Executable Notebooks from Markdown

<details open>
<summary><strong>1. Introduction</strong></summary>

In this lecture, we will learn how to convert AI-generated or manually written **markdown content** into an executable **`.ipynb` Jupyter Notebook**.

This step is **essential** for our AI-Augmented HDL workflow because:

- Most of our lectures and prompts will generate markdown content (via ChatGPT or Gemini).
- Google Colab does **not** accept `.txt` files directly as notebooks.
- Learners need a reliable way to create `.ipynb` files **without local installation**.

Our final goal is:
📄 `.txt` (markdown) ➜ 📓 `.ipynb` notebook ➜ ☁️ GitHub ➜ 🔁 Usable in Colab

</details>

<details open>
<summary><strong>2. Why Markdown-to-Notebook Conversion Is Necessary</strong></summary>

- **AI tools generate markdown**, not executable notebooks.
- **Colab can only run `.ipynb`**, not `.txt`.
- Instead of writing everything manually, we use a **Python script inside Colab** to automate this.
- This step is **repeated in every lecture** in this course, so it is critical to understand.

</details>

<details open>
<summary><strong>3. What You’ll Learn in This Lecture</strong></summary>

- How to paste your markdown content into a Colab notebook
- How to generate a `.ipynb` file from it
- How to upload that notebook to GitHub
- How to create a **reusable Colab link** for future use

</details>

<details open>
<summary><strong>4. Create Your Own Markdown-to-Notebook Converter</strong></summary>

Let us now create our **own converter notebook** that we can reuse for every lecture.

</details>

<details open>
<summary><strong>📋 Step-by-Step Instructions</strong></summary>

1. Open a new Google Colab notebook.
2. Copy and paste the following Python code into the first code cell:

</details>

In [ ]:
from IPython.display import HTML, display
import nbformat as nbf
import uuid
from google.colab import files
from google.colab import output

# Step 1: Show input UI
html_ui = """
<textarea id="markdownInput" style="width:100%;height:300px;" placeholder="Paste your markdown here..."></textarea><br>
<button onclick="google.colab.kernel.invokeFunction('notebook.convert_markdown_to_ipynb', [document.getElementById('markdownInput').value], {})">
Generate Notebook and Download
</button>
"""
display(HTML(html_ui))

# Step 2: Parse and convert markdown
def convert_markdown_to_ipynb(markdown_text):
    lines = markdown_text.strip().split('\n')
    notebook_cells = []

    inside_code_block = False
    code_lines = []
    markdown_buffer = []
    section_title = None

    def flush_markdown():
        nonlocal section_title
        if markdown_buffer:
            content = '\n'.join(markdown_buffer).strip()
            if content:
                if section_title:
                    html_block = f"""<details open>\n<summary><strong>{section_title}</strong></summary>\n\n{content}\n\n</details>"""
                    notebook_cells.append(nbf.v4.new_markdown_cell(html_block))
                    section_title = None  # reset after use
                else:
                    notebook_cells.append(nbf.v4.new_markdown_cell(content))
            markdown_buffer.clear()

    def flush_code():
        if code_lines:
            content = '\n'.join(code_lines).strip()
            if content:
                notebook_cells.append(nbf.v4.new_code_cell(content))
            code_lines.clear()

    for line in lines:
        if line.strip() == '---':
            flush_markdown()
            flush_code()
            section_title = None
            continue

        if line.strip().startswith('```'):
            if inside_code_block:
                flush_code()
                inside_code_block = False
            else:
                flush_markdown()
                inside_code_block = True
            continue

        if inside_code_block:
            code_lines.append(line)
        else:
            if line.strip().startswith('##'):
                flush_markdown()
                section_title = line.strip('# ').strip()
            else:
                markdown_buffer.append(line)

    flush_markdown()
    flush_code()

    nb = nbf.v4.new_notebook()
    nb['cells'] = notebook_cells
    nb['metadata'] = {
        "kernelspec": {
            "display_name": "Python 3",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "name": "python",
            "version": "3.10"
        }
    }

    filename = f"generated_{uuid.uuid4().hex[:6]}.ipynb"
    with open(filename, 'w', encoding='utf-8') as f:
        nbf.write(nb, f)

    files.download(filename)

# Step 3: Register in Colab
output.register_callback('notebook.convert_markdown_to_ipynb', convert_markdown_to_ipynb)

<details open>
<summary><strong>💾 Step 3: Save the Notebook</strong></summary>

- Rename the notebook as:  
  `convert_txt_to_notebook.ipynb`
- Save it by going to:  
  **File > Save a copy in GitHub**
- Choose your repository (e.g., `ai_for_dld_udemy`) and folder (e.g., `colab/`)
- Commit the file with a message like: "Added personal markdown-to-notebook converter"

</details>

<details open>
<summary><strong>🔗 Step 4: Generate the Reusable Colab Link</strong></summary>

Once uploaded, create your personal Colab link using the format:

`[▶️ Open My Markdown Converter](https://colab.research.google.com/github/<your-username>/<your-repo>/blob/main/<path>/<filename>.ipynb)`

This link will open the uploaded notebook directly in Colab — ready for reuse in other lectures.

</details>

<details open>
<summary><strong>✅ 5. Outcome of This Lecture</strong></summary>

By the end of this lecture, you are now able to:

- ✅ Explain why we convert `.txt` markdown files to `.ipynb` notebooks  
- ✅ Use a Colab-based Python script to perform this conversion without any local installation  
- ✅ Save the converted notebook and upload it to GitHub  
- ✅ Generate a Colab-compatible URL that allows anyone to open the notebook instantly

</details>

<details open>
<summary><strong>🙏 6. Thank You</strong></summary>

Thank you for completing this foundational lecture on notebook generation.

In the next lecture, you’ll use this converter to generate your first HDL-based design notebook from a set of AI-generated prompts.

Let’s build your **AI-powered HDL design workflow**, one notebook at a time!

</details>